In [1]:
import pandas as pd
from pandas.api.types import is_string_dtype
import random
from matplotlib import rcParams
import matplotlib.pyplot as plt
import os
import time

In [ ]:
# define output file
Outputpath = 'D:/Powertac/finals-2021/data/'
Outputfilename = 'output' + '.csv'
Outputfilepath = Outputpath + Outpufilename

# define logtool class used to extract game data
logtool_class = 'MktPriceStats'

# Input .csv extracted by powertac logtools
data_input = 'D:/Powertac/finals-2021/data/mktPr1.csv'

In [152]:
if logtool_class == 'MktPriceStats':
    df = pd.read_csv(data_input, header=None)
    # create columns for cleared trades (CT)
    df.columns = ['Timeslot', 'Weekday', 'Hour', 'CT1', 'CT2', 'CT3', 'CT4', 'CT5', 'CT6', 'CT7', 'CT8', 'CT9', 'CT10', 'CT11','CT12', 'CT13', 'CT14', 'CT15', 'CT16', 'CT17', 'CT18', 'CT19', 'CT20','CT21', 'CT22', 'CT23', 'CT24']
    df = df.set_index('Timeslot')
else:
    print('Logtool Class not supported')

# split cleared trades into cleared amounts (CA) and cleared prices (CP)
c_index = 0
for column in df.columns[2:]:
    if is_string_dtype(df[column]) == True:
        df[column] = df[column].str.strip('[]').astype(str)
        c_index = c_index + 1
        CA = 'CA' + str(c_index)
        CP = 'CP' + str(c_index)
        df[[CA,CP]] = df[column].str.split(' ', expand = True)
        df = df.drop(column, axis=1)


df.head()  

,Weekday,Hour,CA1,CP1,CA2,CP2,CA3,CP3,CA4,CP4,...,CA20,CP20,CA21,CP21,CA22,CP22,CA23,CP23,CA24,CP24
Timeslot,,,,,,,,,,,,,,,,,,,,,
363,4,3,14.1679,32.8618,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
364,4,4,5.118,13.5212,1.6868,5.5907,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
365,4,5,1.6493,2.4414,3.4603,6.3035,1.5181,9.0157,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
366,4,6,2.7739,4.0663,1.4844,0.4172,3.1143,3.034,1.3663,38.536,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
367,4,7,2.6573,3.5092,2.4965,37.6041,1.336,1.9046,2.6362,6.2431,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [153]:
# create new Dataframe for the prediction input
data = []
df_main = pd.DataFrame(data)
df_main.index.names = ['Timeslot']
df_main = pd.concat([df_main, df.Weekday], axis=1)

# fill with cleared trades
for column in df.columns[2:]:
    df_main = pd.concat([df_main, df[column]], axis=1)

# add more features:
# cleared trades on hour before
for column in df.columns[2:]:
    name = column + '_H-1'
    last_column = len(df_main.columns)
    df_main.insert(len(df_main.columns), name, df[column].shift(1))

# cleared trades on day before
for column in df.columns[2:]:
    name = column + '_D-1'
    last_column = len(df_main.columns)
    df_main = pd.concat([df_main, df[column].shift(24).rename(name)], axis=1)  
    
# cleared trades on week before
for column in df.columns[2:]:
    name = column + '_W-1'
    last_column = len(df_main.columns)
    df_main = pd.concat([df_main, df[column].shift(168).rename(name)], axis=1)
    
df_main


,Weekday,CA1,CP1,CA2,CP2,CA3,CP3,CA4,CP4,CA5,...,CA20_W-1,CP20_W-1,CA21_W-1,CP21_W-1,CA22_W-1,CP22_W-1,CA23_W-1,CP23_W-1,CA24_W-1,CP24_W-1
Timeslot,,,,,,,,,,,,,,,,,,,,,
363,4,14.1679,32.8618,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
364,4,5.118,13.5212,1.6868,5.5907,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
365,4,1.6493,2.4414,3.4603,6.3035,1.5181,9.0157,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
366,4,2.7739,4.0663,1.4844,0.4172,3.1143,3.034,1.3663,38.536,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
367,4,2.6573,3.5092,2.4965,37.6041,1.336,1.9046,2.6362,6.2431,53.3749,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1737,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5266,46.6667,2.4675,41.113,0.3609,40,0.3839,49.1667,954.1049,81.4491
1738,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5671,42.5,0.474,51.1758,2.2208,36.9198,0.3248,50,975.0534,81.9048
1739,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,52.0302,1,51.826,4.9185,51.4809,1.9987,34.0379,886.3003,47.4719


In [154]:
# create output file from dataframe
df_main.to_csv(Outputfilepath)